# Prompt Engineering Experiment Analysis

This notebook provides interactive analysis of the prompt engineering experiment results.

## Contents
1. Setup and Data Loading
2. Overall Performance Comparison
3. Performance by Question Type
4. Statistical Analysis
5. Improvement vs Baseline
6. Conclusions

## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import json
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Add src to path for imports
sys.path.insert(0, str(Path.cwd().parent / "src"))

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

print("Setup complete!")

In [ ]:
# Load experiment results
results_dir = Path.cwd().parent / "results"

# Load statistics
with open(results_dir / "statistics.json") as f:
    statistics_data = json.load(f)

# Load evaluations
with open(results_dir / "evaluations.json") as f:
    evaluations_data = json.load(f)

# Convert to DataFrames for easier analysis
stats_df = pd.DataFrame(statistics_data).T
stats_df.index.name = "strategy"
stats_df = stats_df.reset_index()

print(f"Loaded statistics for {len(stats_df)} strategies")
stats_df

In [ ]:
# Create evaluation DataFrame
eval_records = []
for strategy, evals in evaluations_data.items():
    for e in evals:
        e["strategy"] = strategy
        eval_records.append(e)

eval_df = pd.DataFrame(eval_records)
print(f"Total evaluations: {len(eval_df)}")
eval_df.head()

## 2. Overall Performance Comparison

In [ ]:
# Bar chart comparing semantic similarity across strategies
fig, ax = plt.subplots(figsize=(10, 6))

strategies = stats_df["strategy"]
similarities = stats_df["mean_semantic_similarity"]
errors = stats_df["std_semantic_similarity"]

colors = sns.color_palette("husl", len(strategies))
bars = ax.bar(strategies, similarities, yerr=errors, capsize=5, color=colors, edgecolor="black")

ax.set_xlabel("Prompting Strategy")
ax.set_ylabel("Mean Semantic Similarity")
ax.set_title("Semantic Similarity Comparison Across Strategies")

for bar, val in zip(bars, similarities):
    ax.annotate(f"{val:.3f}", xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                ha="center", va="bottom", fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Cosine distance comparison
fig, ax = plt.subplots(figsize=(10, 6))

distances = stats_df["mean_cosine_distance"]
distance_errors = stats_df["std_cosine_distance"]

bars = ax.bar(strategies, distances, yerr=distance_errors, capsize=5, 
              color=colors, edgecolor="black")

ax.set_xlabel("Prompting Strategy")
ax.set_ylabel("Mean Cosine Distance")
ax.set_title("Cosine Distance Comparison (Lower is Better)")

for bar, val in zip(bars, distances):
    ax.annotate(f"{val:.3f}", xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                ha="center", va="bottom", fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Exact match rate comparison
fig, ax = plt.subplots(figsize=(10, 6))

match_rates = stats_df["exact_match_rate"] * 100  # Convert to percentage

bars = ax.bar(strategies, match_rates, color=colors, edgecolor="black")

ax.set_xlabel("Prompting Strategy")
ax.set_ylabel("Exact Match Rate (%)")
ax.set_title("Exact Match Rate Comparison")
ax.set_ylim(0, 100)

for bar, val in zip(bars, match_rates):
    ax.annotate(f"{val:.1f}%", xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                ha="center", va="bottom", fontsize=10)

plt.tight_layout()
plt.show()

## 3. Performance by Question Type

In [ ]:
# Load dataset to get question types
with open(Path.cwd().parent / "data" / "raw" / "dataset.json") as f:
    dataset = json.load(f)

question_types = {q["id"]: q["question_type"] for q in dataset}
eval_df["question_type"] = eval_df["question_id"].map(question_types)

eval_df.head()

In [ ]:
# Performance by question type
type_stats = eval_df.groupby(["strategy", "question_type"]).agg({
    "semantic_similarity": ["mean", "std"],
    "cosine_distance": ["mean", "std"],
    "exact_match": "mean"
}).round(3)

type_stats.columns = ["_".join(col).strip() for col in type_stats.columns.values]
type_stats = type_stats.reset_index()
type_stats

In [ ]:
# Grouped bar chart by question type
fig, ax = plt.subplots(figsize=(14, 8))

question_types_list = eval_df["question_type"].unique()
strategies_list = eval_df["strategy"].unique()

x = np.arange(len(question_types_list))
width = 0.2
colors = sns.color_palette("husl", len(strategies_list))

for i, strategy in enumerate(strategies_list):
    values = []
    for qt in question_types_list:
        subset = eval_df[(eval_df["strategy"] == strategy) & (eval_df["question_type"] == qt)]
        values.append(subset["semantic_similarity"].mean())
    
    offset = (i - len(strategies_list)/2 + 0.5) * width
    ax.bar(x + offset, values, width, label=strategy, color=colors[i], edgecolor="black")

ax.set_xlabel("Question Type")
ax.set_ylabel("Mean Semantic Similarity")
ax.set_title("Performance by Question Type and Strategy")
ax.set_xticks(x)
ax.set_xticklabels(question_types_list)
ax.legend(title="Strategy")

plt.tight_layout()
plt.show()

## 4. Statistical Analysis

In [ ]:
# Distribution of cosine distances
fig, ax = plt.subplots(figsize=(12, 6))

for strategy in strategies_list:
    subset = eval_df[eval_df["strategy"] == strategy]
    ax.hist(subset["cosine_distance"], bins=15, alpha=0.6, label=strategy, edgecolor="black")

ax.set_xlabel("Cosine Distance")
ax.set_ylabel("Frequency")
ax.set_title("Distribution of Cosine Distances by Strategy")
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Box plot of semantic similarity
fig, ax = plt.subplots(figsize=(10, 6))

sns.boxplot(data=eval_df, x="strategy", y="semantic_similarity", ax=ax, palette="husl")

ax.set_xlabel("Prompting Strategy")
ax.set_ylabel("Semantic Similarity")
ax.set_title("Semantic Similarity Distribution by Strategy")

plt.tight_layout()
plt.show()

In [ ]:
# Summary statistics table
summary = eval_df.groupby("strategy").agg({
    "semantic_similarity": ["count", "mean", "std", "min", "max"],
    "cosine_distance": ["mean", "std"],
    "exact_match": ["sum", "mean"]
})

summary.columns = ["_".join(col) for col in summary.columns]
summary = summary.round(4)
summary

## 5. Improvement vs Baseline

In [ ]:
# Calculate improvement percentages vs baseline
baseline_similarity = stats_df[stats_df["strategy"] == "baseline"]["mean_semantic_similarity"].values[0]

improvements = []
for _, row in stats_df.iterrows():
    if row["strategy"] != "baseline":
        improvement = ((row["mean_semantic_similarity"] - baseline_similarity) / baseline_similarity) * 100
        improvements.append({
            "strategy": row["strategy"],
            "improvement_pct": improvement
        })

improvement_df = pd.DataFrame(improvements)
improvement_df

In [ ]:
# Improvement chart
fig, ax = plt.subplots(figsize=(10, 6))

colors = ["green" if x >= 0 else "red" for x in improvement_df["improvement_pct"]]
bars = ax.bar(improvement_df["strategy"], improvement_df["improvement_pct"], 
              color=colors, edgecolor="black")

ax.axhline(y=0, color="black", linestyle="-", linewidth=0.5)
ax.set_xlabel("Prompting Strategy")
ax.set_ylabel("Improvement vs Baseline (%)")
ax.set_title("Performance Improvement Compared to Baseline Strategy")

for bar, val in zip(bars, improvement_df["improvement_pct"]):
    va = "bottom" if val >= 0 else "top"
    ax.annotate(f"{val:+.1f}%", xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                ha="center", va=va, fontsize=10, fontweight="bold")

plt.tight_layout()
plt.show()

In [ ]:
# Heatmap of performance metrics
metrics_for_heatmap = stats_df.set_index("strategy")[[
    "mean_semantic_similarity", 
    "mean_cosine_distance",
    "exact_match_rate"
]].T

fig, ax = plt.subplots(figsize=(10, 4))

sns.heatmap(metrics_for_heatmap, annot=True, fmt=".3f", cmap="RdYlGn", ax=ax)
ax.set_title("Performance Metrics Heatmap")
ax.set_ylabel("Metric")
ax.set_xlabel("Strategy")

plt.tight_layout()
plt.show()

## 6. Conclusions

In [ ]:
# Rank strategies by performance
ranking = stats_df.copy()
ranking["rank_similarity"] = ranking["mean_semantic_similarity"].rank(ascending=False)
ranking["rank_distance"] = ranking["mean_cosine_distance"].rank(ascending=True)
ranking["rank_exact_match"] = ranking["exact_match_rate"].rank(ascending=False)
ranking["overall_rank"] = (ranking["rank_similarity"] + ranking["rank_distance"] + ranking["rank_exact_match"]) / 3

ranking = ranking.sort_values("overall_rank")
print("Strategy Ranking (Best to Worst):")
print("=" * 50)
for i, (_, row) in enumerate(ranking.iterrows(), 1):
    print(f"{i}. {row['strategy'].upper()}")
    print(f"   Semantic Similarity: {row['mean_semantic_similarity']:.3f}")
    print(f"   Cosine Distance:     {row['mean_cosine_distance']:.3f}")
    print(f"   Exact Match Rate:    {row['exact_match_rate']:.1%}")
    print()

In [ ]:
# Final summary
print("\n" + "=" * 60)
print("EXPERIMENT SUMMARY")
print("=" * 60)
print(f"\nTotal questions evaluated: {len(dataset)}")
print(f"Strategies compared: {len(stats_df)}")
print(f"\nBest performing strategy: {ranking.iloc[0]['strategy'].upper()}")
print(f"  - Mean Semantic Similarity: {ranking.iloc[0]['mean_semantic_similarity']:.3f}")
print(f"  - Mean Cosine Distance: {ranking.iloc[0]['mean_cosine_distance']:.3f}")
print(f"  - Exact Match Rate: {ranking.iloc[0]['exact_match_rate']:.1%}")

if len(improvement_df) > 0:
    best_improvement = improvement_df.loc[improvement_df["improvement_pct"].idxmax()]
    print(f"\nBest improvement over baseline: {best_improvement['strategy'].upper()}")
    print(f"  - Improvement: {best_improvement['improvement_pct']:+.1f}%")

print("\n" + "=" * 60)